In [1]:
import pandas as pd
import csv

In [2]:
# Load the CSV file
FA = '../FAresults.xlsx'
CV = '../CVresults.xlsx'
static2SSP = '../static2SSPresults.xlsx'
rolling2SSP = '../rolling2SSPresults.xlsx'
WS = '../WSresults.xlsx'
naiveWS = '../naiveWSresults.xlsx'

df_FA = pd.read_excel(FA)
df_CV = pd.read_excel(CV)
df_s2SSP = pd.read_excel(static2SSP)
df_r2SSP = pd.read_excel(rolling2SSP)
df_WS = pd.read_excel(WS)
df_naiveWS = pd.read_excel(naiveWS)

In [3]:
if df_CV.shape[0] != df_FA.shape[0] or df_CV.shape[0] != df_s2SSP.shape[0] or df_CV.shape[0] != df_r2SSP.shape[0] or df_CV.shape[0] != df_WS.shape[0]:
    print("ERROR! # of rows in the data frames are inconsistent!")
else:
    print("df_CV.shape = ", df_CV.shape)

df_CV.shape =  (108, 12)


In [4]:
# Do separate tables for different instance_option (-1: deterministic, 0: random) and cost_structure (0: old,1: safetime)

In [5]:
def createTable(i_option, c_structure, outputfile):
    df_CV_0 = df_CV[(df_CV['instance_option'] == i_option) & (df_CV['cost_structure'] == c_structure)]
    df_FA_0 = df_FA[(df_FA['instance_option'] == i_option) & (df_FA['cost_structure'] == c_structure)]
    df_s2SSP_0 = df_s2SSP[(df_s2SSP['instance_option'] == i_option) & (df_s2SSP['cost_structure'] == c_structure)]
    df_r2SSP_0 = df_r2SSP[(df_r2SSP['instance_option'] == i_option) & (df_r2SSP['cost_structure'] == c_structure)]
    df_WS_0 = df_WS[(df_WS['instance_option'] == i_option) & (df_WS['cost_structure'] == c_structure)]
    df_CV_0.reset_index(drop=True, inplace=True)
    df_FA_0.reset_index(drop=True, inplace=True)
    df_s2SSP_0.reset_index(drop=True, inplace=True)
    df_r2SSP_0.reset_index(drop=True, inplace=True)
    df_WS_0.reset_index(drop=True, inplace=True)
    if c_structure == 0:
        # table macros for this case
        tauList = [0, 0.5, 5]
        NiList = [3,6,9]
        NjList = [10,20,30]
    elif c_structure == 1:
        tauList = [2, 5, 10]
        NiList = [3,6,9]
        NjList = [10,20,30]
    
    # Create a LaTeX table string with the specified columns and multi-row header
    latex_table_custom = '';
    
    for i1 in range(len(tauList)):
        for i2 in range(len(NiList)):
            for i3 in range(len(NjList)):
                idx = i2*len(NjList)*len(tauList)+i3*len(tauList)+i1
                CV_val = df_CV_0["obj"][idx] 
                CV_CI = df_CV_0["CI"][idx] 
                FA_LB = df_FA_0["LB"][idx]
                FA_obj = df_FA_0["obj"][idx]
                FA_CI = df_FA_0["CI"][idx]
                FA_time = df_FA_0["train_time"][idx]
                FA_gap = int((df_FA_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                WS_time = df_WS_0["train_time"][idx]
                WS_gap = int((df_WS_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                r2SSP_gap = int((df_r2SSP_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                s2SSP_time = df_s2SSP_0["train_time"][idx]
                s2SSP_gap = int((df_s2SSP_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)


                if i2 == 0:
                    if i3 == 0:
                        latex_table_custom += f"{tauList[i1]} & {NiList[i2]} $\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\% \\\\ \n"
                    else:
                        latex_table_custom += f" &  $\quad\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
                else:
                    if i3 == 0:
                        latex_table_custom += f" & {NiList[i2]}  $\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
                    else:
                        latex_table_custom += f" &  $\quad\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
                idx += 1;
        if i1 != len(tauList)-1:
            latex_table_custom += "\\hline\n"

    print(latex_table_custom)        
    # Save to a new .tex file
    with open(outputfile, 'w') as f:
        f.write(latex_table_custom)

In [6]:
createTable(-1,0,'Table-synthetic-gap-summary1.tex')
createTable(0,0,'Table-synthetic-gap-summary2.tex')
#createTable(-1,1,'Table-synthetic-gap-summary3.tex')
#createTable(0,1,'Table-synthetic-gap-summary4.tex')

0 & 3 $\quad$ 10 && 710.8 & 93.0 && 1693.2 & 1646.4$\pm$147.5 & 20.1 & 131\% && 2.8 & 131\% && 131\% && 0.0 & 753\% \\ 
 &  $\quad\quad$ 20 && 1421.6 & 179.0 && 3175.3 & 3061.8$\pm$265.8 & 22.6 & 115\% && 4.6 & 115\% && 115\% && 0.0 & 696\%  \\ 
 &  $\quad\quad$ 30 && 2123.2 & 267.0 && 4859.6 & 4696.2$\pm$407.4 & 24.3 & 121\% && 6.2 & 121\% && 121\% && 0.1 & 707\%  \\ 
 & 6  $\quad$ 10 && 1093.4 & 286.9 && 1953.1 & 1965.2$\pm$299.4 & 28.1 & 79\% && 4.2 & 79\% && 79\% && 0.0 & 462\%  \\ 
 &  $\quad\quad$ 20 && 1645.7 & 294.4 && 3315.3 & 3202.7$\pm$337.8 & 31.1 & 94\% && 6.7 & 94\% && 94\% && 0.1 & 603\%  \\ 
 &  $\quad\quad$ 30 && 2542.8 & 477.8 && 5205.7 & 5017.4$\pm$548.5 & 38.1 & 97\% && 9.2 & 97\% && 97\% && 0.1 & 581\%  \\ 
 & 9  $\quad$ 10 && 908.8 & 202.0 && 1807.3 & 1801.0$\pm$224.0 & 29.6 & 98\% && 5.6 & 98\% && 98\% && 0.0 & 579\%  \\ 
 &  $\quad\quad$ 20 && 1458.4 & 194.4 && 3249.1 & 3058.3$\pm$267.0 & 39.7 & 109\% && 8.8 & 109\% && 109\% && 0.1 & 700\%  \\ 
 &  $\quad\quad$ 

In [7]:
def createCSV4Figure(i_option, c_structure, outputfile):
    df_CV_0 = df_CV[(df_CV['instance_option'] == i_option) & (df_CV['cost_structure'] == c_structure)]
    df_FA_0 = df_FA[(df_FA['instance_option'] == i_option) & (df_FA['cost_structure'] == c_structure)]
    df_s2SSP_0 = df_s2SSP[(df_s2SSP['instance_option'] == i_option) & (df_s2SSP['cost_structure'] == c_structure)]
    df_r2SSP_0 = df_r2SSP[(df_r2SSP['instance_option'] == i_option) & (df_r2SSP['cost_structure'] == c_structure)]
    df_WS_0 = df_WS[(df_WS['instance_option'] == i_option) & (df_WS['cost_structure'] == c_structure)]
    df_CV_0.reset_index(drop=True, inplace=True)
    df_FA_0.reset_index(drop=True, inplace=True)
    df_s2SSP_0.reset_index(drop=True, inplace=True)
    df_r2SSP_0.reset_index(drop=True, inplace=True)
    df_WS_0.reset_index(drop=True, inplace=True)
    if c_structure == 0:
        # table macros for this case
        tauList = [0, 0.5, 5]
        NiList = [3,6,9]
        NjList = [10,20,30]
    elif c_structure == 1:
        tauList = [2, 5, 10]
        NiList = [3,6,9]
        NjList = [10,20,30]
    
    with open(outputfile, 'a') as myfile:
        writer = csv.writer(myfile, delimiter =',')
                    
        for i1 in range(len(tauList)):
            for i2 in range(len(NiList)):
                for i3 in range(len(NjList)):
                    idx = i2*len(NjList)*len(tauList)+i3*len(tauList)+i1
                    FA_gap = int((df_FA_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                    WS_gap = int((df_WS_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                    r2SSP_gap = int((df_r2SSP_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                    s2SSP_gap = int((df_s2SSP_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                    writer.writerow([tauList[i1],NiList[i2],NjList[i3],FA_gap,WS_gap,r2SSP_gap,s2SSP_gap])

In [8]:
createCSV4Figure(-1,0,'for-Figure-synthetic-gap-1.csv')
createCSV4Figure(0,0,'for-Figure-synthetic-gap-2.csv')
#createCSV4Figure(-1,1,'for-Figure-synthetic-gap-3.csv')
#createCSV4Figure(0,1,'for-Figure-synthetic-gap-4.csv')

In [9]:
def createTable2(i_option, outputfile):
    df_CV_0 = df_CV[(df_CV['instance_option'] == i_option) & (df_CV['cost_structure'] == 1)]
    df_FA_0 = df_FA[(df_FA['instance_option'] == i_option) & (df_FA['cost_structure'] == 1)]
    df_s2SSP_0 = df_s2SSP[(df_s2SSP['instance_option'] == i_option) & (df_s2SSP['cost_structure'] == 1)]
    df_r2SSP_0 = df_r2SSP[(df_r2SSP['instance_option'] == i_option) & (df_r2SSP['cost_structure'] == 1)]
    df_WS_0 = df_WS[(df_WS['instance_option'] == i_option) & (df_WS['cost_structure'] == 1)]
    df_naiveWS_0 = df_naiveWS[(df_naiveWS['instance_option'] == i_option)]
    df_CV_0.reset_index(drop=True, inplace=True)
    df_FA_0.reset_index(drop=True, inplace=True)
    df_s2SSP_0.reset_index(drop=True, inplace=True)
    df_r2SSP_0.reset_index(drop=True, inplace=True)
    df_WS_0.reset_index(drop=True, inplace=True)
    df_naiveWS_0.reset_index(drop=True, inplace=True)
                              
    df_CV_1 = df_CV[(df_CV['instance_option'] == i_option) & (df_CV['cost_structure'] == 0)]
    df_FA_1 = df_FA[(df_FA['instance_option'] == i_option) & (df_FA['cost_structure'] == 0)]
    df_s2SSP_1 = df_s2SSP[(df_s2SSP['instance_option'] == i_option) & (df_s2SSP['cost_structure'] == 0)]
    df_r2SSP_1 = df_r2SSP[(df_r2SSP['instance_option'] == i_option) & (df_r2SSP['cost_structure'] == 0)]
    df_WS_1 = df_WS[(df_WS['instance_option'] == i_option) & (df_WS['cost_structure'] == 0)]
    df_CV_1.reset_index(drop=True, inplace=True)
    df_FA_1.reset_index(drop=True, inplace=True)
    df_s2SSP_1.reset_index(drop=True, inplace=True)
    df_r2SSP_1.reset_index(drop=True, inplace=True)
    df_WS_1.reset_index(drop=True, inplace=True)

    tauList = [1, 2, 5, 10]
    NiList = [3,6,9]
    NjList = [10,20,30]
    
    # Create a LaTeX table string with the specified columns and multi-row header
    latex_table_custom = '';
    
    for i1 in range(len(tauList)):
        if i1 == 0:
            for i2 in range(len(NiList)):
                for i3 in range(len(NjList)):
                    idx = i2*len(NjList)*(len(tauList)-1)+i3*(len(tauList)-1)
                    CV_val = df_CV_1["obj"][idx] 
                    CV_CI = df_CV_1["CI"][idx] 
                    FA_obj = df_FA_1["obj"][idx]
                    FA_CI = df_FA_1["CI"][idx]
                    FA_LB = df_FA_1["LB"][idx]
                    FA_time = df_FA_1["train_time"][idx]
                    FA_gap = int((df_FA_1["obj"][idx]-df_CV_1["obj"][idx])/df_CV_1["obj"][idx]*100)
                    WS_time = df_WS_1["train_time"][idx]
                    WS_gap = int((df_WS_1["obj"][idx]-df_CV_1["obj"][idx])/df_CV_1["obj"][idx]*100)
                    r2SSP_gap = int((df_r2SSP_1["obj"][idx]-df_CV_1["obj"][idx])/df_CV_1["obj"][idx]*100)
                    s2SSP_time = df_s2SSP_1["train_time"][idx]
                    s2SSP_gap = int((df_s2SSP_1["obj"][idx]-df_CV_1["obj"][idx])/df_CV_1["obj"][idx]*100)
    
    
                    idx2 = i2*len(NjList)*(len(tauList))+i3*(len(tauList))
                    naiveWS_gap = int((df_naiveWS_0["obj"][idx2]-df_CV_1["obj"][idx])/df_CV_1["obj"][idx]*100)
                              
                    if i2 == 0:
                        if i3 == 0:
                            latex_table_custom += f"{tauList[i1]} & {NiList[i2]} $\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% &&  {naiveWS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\% \\\\ \n"
                        else:
                            latex_table_custom += f" &  $\quad\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% &&  {naiveWS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
                    else:
                        if i3 == 0:
                            latex_table_custom += f" & {NiList[i2]}  $\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% &&  {naiveWS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
                        else:
                            latex_table_custom += f" &  $\quad\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% && {naiveWS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
        else:
            for i2 in range(len(NiList)):
                for i3 in range(len(NjList)):
                    idx = i2*len(NjList)*(len(tauList)-1)+i3*(len(tauList)-1)+i1-1
                    CV_val = df_CV_0["obj"][idx] 
                    CV_CI = df_CV_0["CI"][idx] 
                    FA_LB = df_FA_0["LB"][idx]
                    FA_CI = df_FA_0["CI"][idx]
                    FA_obj = df_FA_0["obj"][idx]
                    FA_time = df_FA_0["train_time"][idx]
                    FA_gap = int((df_FA_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                    WS_time = df_WS_0["train_time"][idx]
                    WS_gap = int((df_WS_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                    r2SSP_gap = int((df_r2SSP_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                    s2SSP_time = df_s2SSP_0["train_time"][idx]
                    s2SSP_gap = int((df_s2SSP_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)

                    idx2 = i2*len(NjList)*(len(tauList))+i3*(len(tauList))+i1
                    naiveWS_gap = int((df_naiveWS_0["obj"][idx2]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)

                    if i2 == 0:
                        if i3 == 0:
                            latex_table_custom += f"{tauList[i1]} & {NiList[i2]} $\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% && {naiveWS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\% \\\\ \n"
                        else:
                            latex_table_custom += f" &  $\quad\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% &&  {naiveWS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
                    else:
                        if i3 == 0:
                            latex_table_custom += f" & {NiList[i2]}  $\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% &&  {naiveWS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
                        else:
                            latex_table_custom += f" &  $\quad\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_LB,1)} & {round(FA_obj,1)}$\pm${round(FA_CI,1)} & {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% &&  {naiveWS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
        if i1 != len(tauList)-1:
            latex_table_custom += "\\hline\n"

    print(latex_table_custom)        
    # Save to a new .tex file
    with open(outputfile, 'w') as f:
        f.write(latex_table_custom)

In [10]:
def createCSV4Figure2(i_option, outputfile):
    df_CV_0 = df_CV[(df_CV['instance_option'] == i_option) & (df_CV['cost_structure'] == 1)]
    df_FA_0 = df_FA[(df_FA['instance_option'] == i_option) & (df_FA['cost_structure'] == 1)]
    df_s2SSP_0 = df_s2SSP[(df_s2SSP['instance_option'] == i_option) & (df_s2SSP['cost_structure'] == 1)]
    df_r2SSP_0 = df_r2SSP[(df_r2SSP['instance_option'] == i_option) & (df_r2SSP['cost_structure'] == 1)]
    df_WS_0 = df_WS[(df_WS['instance_option'] == i_option) & (df_WS['cost_structure'] == 1)]
    df_naiveWS_0 = df_naiveWS[(df_naiveWS['instance_option'] == i_option)]
    df_CV_0.reset_index(drop=True, inplace=True)
    df_FA_0.reset_index(drop=True, inplace=True)
    df_WS_0.reset_index(drop=True, inplace=True)
    df_s2SSP_0.reset_index(drop=True, inplace=True)
    df_r2SSP_0.reset_index(drop=True, inplace=True)
    df_naiveWS_0.reset_index(drop=True, inplace=True)
                              
    df_CV_1 = df_CV[(df_CV['instance_option'] == i_option) & (df_CV['cost_structure'] == 0)]
    df_FA_1 = df_FA[(df_FA['instance_option'] == i_option) & (df_FA['cost_structure'] == 0)]
    df_s2SSP_1 = df_s2SSP[(df_s2SSP['instance_option'] == i_option) & (df_s2SSP['cost_structure'] == 0)]
    df_r2SSP_1 = df_r2SSP[(df_r2SSP['instance_option'] == i_option) & (df_r2SSP['cost_structure'] == 0)]
    df_WS_1 = df_WS[(df_WS['instance_option'] == i_option) & (df_WS['cost_structure'] == 0)]
    df_CV_1.reset_index(drop=True, inplace=True)
    df_FA_1.reset_index(drop=True, inplace=True)
    df_WS_1.reset_index(drop=True, inplace=True)
    df_s2SSP_1.reset_index(drop=True, inplace=True)
    df_r2SSP_1.reset_index(drop=True, inplace=True)

    tauList = [1, 2, 5, 10]
    NiList = [3,6,9]
    NjList = [10,20,30]
    
    with open(outputfile, 'a') as myfile:
        writer = csv.writer(myfile, delimiter =',')
                    
        for i1 in range(len(tauList)):
            if i1 == 0:
                for i2 in range(len(NiList)):
                    for i3 in range(len(NjList)):
                        idx = i2*len(NjList)*(len(tauList)-1)+i3*(len(tauList)-1)
                        FA_gap = int((df_FA_1["obj"][idx]-df_CV_1["obj"][idx])/df_CV_1["obj"][idx]*100)
                        WS_gap = int((df_WS_1["obj"][idx]-df_CV_1["obj"][idx])/df_CV_1["obj"][idx]*100)
                        r2SSP_gap = int((df_r2SSP_1["obj"][idx]-df_CV_1["obj"][idx])/df_CV_1["obj"][idx]*100)
                        s2SSP_gap = int((df_s2SSP_1["obj"][idx]-df_CV_1["obj"][idx])/df_CV_1["obj"][idx]*100)
                        idx2 = i2*len(NjList)*(len(tauList))+i3*(len(tauList))
                        naiveWS_gap = int((df_naiveWS_0["obj"][idx2]-df_CV_1["obj"][idx])/df_CV_1["obj"][idx]*100)
                        writer.writerow([tauList[i1],NiList[i2],NjList[i3],FA_gap,WS_gap,naiveWS_gap,r2SSP_gap,s2SSP_gap])
            else:
                for i2 in range(len(NiList)):
                    for i3 in range(len(NjList)):
                        idx = i2*len(NjList)*(len(tauList)-1)+i3*(len(tauList)-1)+i1-1
                        CV_val = df_CV_0["obj"][idx] 
                        CV_CI = df_CV_0["CI"][idx] 
                        FA_time = df_FA_0["train_time"][idx]
                        FA_gap = int((df_FA_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                        WS_time = df_WS_0["train_time"][idx]
                        WS_gap = int((df_WS_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                        r2SSP_gap = int((df_r2SSP_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                        s2SSP_time = df_s2SSP_0["train_time"][idx]
                        s2SSP_gap = int((df_s2SSP_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)

                        idx2 = i2*len(NjList)*(len(tauList))+i3*(len(tauList))+i1
                        naiveWS_gap = int((df_naiveWS_0["obj"][idx2]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                        writer.writerow([tauList[i1],NiList[i2],NjList[i3],FA_gap,WS_gap,naiveWS_gap,r2SSP_gap,s2SSP_gap])

In [11]:
createTable2(-1,'Table-synthetic-gap-summary3.tex')
createTable2(0,'Table-synthetic-gap-summary4.tex')

1 & 3 $\quad$ 10 && 710.8 & 93.0 && 1693.2 & 1646.4$\pm$147.5 & 20.1 & 131\% && 2.8 & 131\% &&  350\% && 131\% && 0.0 & 753\% \\ 
 &  $\quad\quad$ 20 && 1421.6 & 179.0 && 3175.3 & 3061.8$\pm$265.8 & 22.6 & 115\% && 4.6 & 115\% &&  331\% && 115\% && 0.0 & 696\%  \\ 
 &  $\quad\quad$ 30 && 2123.2 & 267.0 && 4859.6 & 4696.2$\pm$407.4 & 24.3 & 121\% && 6.2 & 121\% &&  337\% && 121\% && 0.1 & 707\%  \\ 
 & 6  $\quad$ 10 && 1093.4 & 286.9 && 1953.1 & 1965.2$\pm$299.4 & 28.1 & 79\% && 4.2 & 79\% &&  216\% && 79\% && 0.0 & 462\%  \\ 
 &  $\quad\quad$ 20 && 1645.7 & 294.4 && 3315.3 & 3202.7$\pm$337.8 & 31.1 & 94\% && 6.7 & 94\% && 281\% && 94\% && 0.1 & 603\%  \\ 
 &  $\quad\quad$ 30 && 2542.8 & 477.8 && 5205.7 & 5017.4$\pm$548.5 & 38.1 & 97\% && 9.2 & 97\% && 276\% && 97\% && 0.1 & 581\%  \\ 
 & 9  $\quad$ 10 && 908.8 & 202.0 && 1807.3 & 1801.0$\pm$224.0 & 29.6 & 98\% && 5.6 & 98\% &&  267\% && 98\% && 0.0 & 579\%  \\ 
 &  $\quad\quad$ 20 && 1458.4 & 194.4 && 3249.1 & 3058.3$\pm$267.0 & 39.7 &

In [12]:
createCSV4Figure2(-1,'for-Figure-synthetic-gap-3.csv')
createCSV4Figure2(0,'for-Figure-synthetic-gap-4.csv')